# 0. Intro

### On my mind (in case I wake up with dementia :D)
I want to build an RSA. For this I have several parameters to consider, so I have to do this per subject, contrast (of all tasks), per hemisphere.

If there is more than one session of the same task_contrast (space&subject&task&contrast&hemisphere), I guess I should average them?

I have the Glasser atlas in fsaverage space (in which I want to work) and the parcels that I want to use (only Frontoparietal). I need to map the voxels from the atlas to all parcels, and then only keep the FPN parcels.

With the info on which vertices to include (FPN) I want to shorten the contrast maps from whole brain activation to activation of FPN vertices. Then I will separate those to get beta maps per parcel, per contrast.

Question: When looking at parcel X and contrast Y, how do I get from a matrix of *vertices x task* to *task x task* correlation matrix?

At this point I am still within a single subject.

In [1]:
import os
import nibabel as nib
from nibabel.freesurfer.io import read_annot
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr

base_dir = 'data/contrast_maps/' # find the contrast maps here
output_dir = 'data/output'
atlas_dir = 'data/atlas'


# 1. Load atlas and extract FPN parcels

## 1.1 Glasser: vertex-to-parcel mappings
1. **Vertex Labels**: Each vertex on the cortical surface mesh is assigned a label indicating the region or parcel it belongs to (No.: 2 x 163842).
2. **Color Table**: A mapping of labels to colors, which helps in visualizing the different regions on the cortical surface.
3. **Region Names**: Names of the regions or parcels corresponding to the labels (No.: 2 x 181).


In [2]:
# Load the annotation files
lh_annot_file = os.path.join(atlas_dir, 'lh.HCP-MMP1.annot')
rh_annot_file = os.path.join(atlas_dir, 'rh.HCP-MMP1.annot')

# Read the annotation files
labels_lh, ctab_lh, names_lh = read_annot(lh_annot_file)
labels_rh, ctab_rh, names_rh = read_annot(rh_annot_file)

# Overview on annot (for left hemisphere, but equally applies to right hemisphere)
# Labels: Length of labels: 163842 (= amount of vertices in the left hemisphere in fsaverage)
print('Length of left hemisphere labels:', len(labels_lh))
print('First 15 labels of left hemisphere:', labels_lh[:15])

# Parcels: Number of unique labels: 181 (= amount of parcels in the left hemisphere)
unique_labels_lh = np.unique(labels_lh)
print('Amount of unique labels in left hemisphere:', len(unique_labels_lh))

Loading Glasser atlas...


In [3]:
# Create vertex-to-parcel mappings in Glasser atlas
vertices_lh = np.arange(len(labels_lh))
vertices_rh = np.arange(len(labels_rh))
lh_parcel_mapping = {vertex: names_lh[label] for vertex, label in zip(vertices_lh, labels_lh)}
rh_parcel_mapping = {vertex: names_rh[label] for vertex, label in zip(vertices_rh, labels_rh)}

# Print the first mappings
print("Left Hemisphere Parcel Mapping (first 5 entries):")
print({k: lh_parcel_mapping[k] for k in list(lh_parcel_mapping)[:5]})

print("Right Hemisphere Parcel Mapping (first 5 entries):")
print({k: rh_parcel_mapping[k] for k in list(rh_parcel_mapping)[:5]})

Left Hemisphere Parcel Mapping (first 5 entries):
{np.int64(0): np.bytes_(b'L_6d_ROI'), np.int64(1): np.bytes_(b'L_VIP_ROI'), np.int64(2): np.bytes_(b'L_24dv_ROI'), np.int64(3): np.bytes_(b'L_9-46d_ROI'), np.int64(4): np.bytes_(b'L_43_ROI')}
Right Hemisphere Parcel Mapping (first 5 entries):
{np.int64(0): np.bytes_(b'R_6mp_ROI'), np.int64(1): np.bytes_(b'R_AIP_ROI'), np.int64(2): np.bytes_(b'R_1_ROI'), np.int64(3): np.bytes_(b'R_46_ROI'), np.int64(4): np.bytes_(b'R_p32pr_ROI')}


## 1.2 Network Partition: FPN parcels-vertices

In [4]:
# Load the text file containing labels of parcels
network_partition_path = os.path.join(atlas_dir, 'CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR_LabelKey.txt')
network_partition = pd.read_csv(network_partition_path, sep='\t')

# Give out overall amount of parcels
print(f"Total number of parcels (= unique labels): {len(network_partition)}")
print(f"Number of unique networks (l&r): {network_partition['NETWORK'].nunique()}")
print(f"Number of unique glasser labels (l&r): {network_partition['GLASSERLABELNAME'].nunique()}")
network_partition.head()

Total number of parcels (= unique labels): 718
Number of unique networks (l&r): 12
Number of unique glasser labels (l&r): 360


In [5]:
# Extract parcels of FPN
fpn_parcels = network_partition[network_partition['NETWORK'] == 'Frontoparietal']

# Give out amount of FPN parcels (98, where 46 are in the left hemisphere and 50 in the right hemisphere)
print(f"Number of labels within FPN: {len(fpn_parcels)}")
print(f"Number of unique glasser labels from FPN: {fpn_parcels['GLASSERLABELNAME'].nunique()}")
fpn_parcels.head()

Number of labels within FPN: 98
Number of unique glasser labels from FPN: 50


In [6]:
# Inspect parcels from left and right hemisphere of FPN separately
fpn_parcels_lh = fpn_parcels[fpn_parcels['HEMISPHERE'] == 'L']
fpn_parcels_rh = fpn_parcels[fpn_parcels['HEMISPHERE'] == 'R']
fpn_parcels_both = fpn_parcels[fpn_parcels['HEMISPHERE'] == 'LR']
print(f"Number of Glasser parcels from left FPN: {fpn_parcels_lh['GLASSERLABELNAME'].nunique()}")
print(f"Number of Glasser parcels from right FPN: {fpn_parcels_rh['GLASSERLABELNAME'].nunique()}")
print(f"Number of Glasser parcels from FPN inbetween hemisphere: {fpn_parcels_both['GLASSERLABELNAME'].nunique()}")
fpn_parcels_rh.head()

Number of Glasser parcels from left FPN: 22
Number of Glasser parcels from right FPN: 28
Number of Glasser parcels from FPN inbetween hemisphere: 0


In [7]:
import pandas as pd

# Use atlas annotation (lh_parcel_mapping & rh_parcel_mapping) and only keep parcels of FPN
# Use GLASSERLABELNAME in fpn_parcels to match with lh_parcel_mapping & rh_parcel_mapping

# Left hemisphere: all vertices assigned to parcels of FPN
fpn_parcels_lh_names = list(fpn_parcels_lh['GLASSERLABELNAME'].dropna())
fpn_parcels_lh_mapping = {vertex: lh_parcel_mapping[vertex] for vertex in vertices_lh if lh_parcel_mapping[vertex].decode('utf-8') in fpn_parcels_lh_names}

# Right hemisphere: all vertices assigned to parcels of FPN
fpn_parcels_rh_names = list(fpn_parcels_rh['GLASSERLABELNAME'].dropna())
fpn_parcels_rh_mapping = {vertex: rh_parcel_mapping[vertex] for vertex in vertices_rh if rh_parcel_mapping[vertex].decode('utf-8') in fpn_parcels_rh_names}

# Check if the number of parcels in the mapping is correct
print(f"Number of vertices within parcels of FPN in left hemisphere mapping: {len(fpn_parcels_lh_mapping)}")
print(f"Number of vertices within parcels of FPN in right hemisphere mapping: {len(fpn_parcels_rh_mapping)}")

# Visualization of vertex - parcel mapping
# Convert the mappings to DataFrames for better visualization
lh_mapping_df = pd.DataFrame(list(fpn_parcels_lh_mapping.items()), columns=['Vertex', 'Parcel'])
rh_mapping_df = pd.DataFrame(list(fpn_parcels_rh_mapping.items()), columns=['Vertex', 'Parcel'])

# Display a few entries of the mappings (here left hemisphere)
print("Left Hemisphere Parcel Mapping:")
lh_mapping_df

Number of vertices within parcels of FPN in left hemisphere mapping: 18361
Number of vertices within parcels of FPN in right hemisphere mapping: 22966
Left Hemisphere Parcel Mapping:


# 2. Load contrast data

In [8]:
# Parameters
space = 'fsaverage7' # 'fsaverage5' or 'fsaverage7'

subject = '01'
session = '00'
task = 'ArchiStandard'
contrast = 'audio_computation'
hemisphere = 'lh' # 'lh' or 'rh'

In [9]:
# Load gifti and output shape -> amount of vertices / data points (163842)

file_path = [f"{base_dir}/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_task-{task}_dir-ffx_space-{space}_hemi-{hemisphere}_ZMap-{contrast}.gii"]
img = nib.load(file_path[0])
data = np.array([darray.data for darray in img.darrays])
print(f"Shape of data: {data.shape}")
data

In [10]:
### Now turn into pipeline so that you can loop over all subjects, sessions, tasks, contrasts and hemispheres

# Needed to come up with RSMs (contrast x contrast)

# Initialize lists to store results
results = []

# Define subjects, sessions, tasks, contrasts, and hemispheres
subjects = ['01', '02']  # Add more subjects as needed
sessions = ['00', '01']   # Add more sessions as needed
tasks = ['ArchiStandard']  # Add more tasks as needed
contrasts = ['audio_computation']  # Add more contrasts as needed
hemispheres = ['lh', 'rh']  # Left and right hemispheres

# Iterate through all combinations
for subject in subjects:
    for session in sessions:
        for task in tasks:
            for contrast in contrasts:
                for hemisphere in hemispheres:
                    # Load data for the current combination
                    file_path = [f"{base_dir}/sub-{subject}/ses-{session}/sub-{subject}_ses-{session}_task-{task}_dir-ffx_space-{space}_hemi-{hemisphere}_ZMap-{contrast}.gii"]
                    img = nib.load(file_path[0])
                    data = np.array([darray.data for darray in img.darrays])
                    # Process data and calculate correlation matrices here
                    # Append results to the results list
                    results.append((subject, session, task, contrast, hemisphere, data))

# Save results to output directory
output_file = os.path.join(output_dir, 'results.npy')
np.save(output_file, results)
print(f"Results saved to {output_file}")